1. First we will load in the training and test data, which has been prepared via band_pass etc to reduce background noise and has had the mfcc, rms, and spec centroid taken as features.

In [1]:
import numpy as np
import os
import time
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC  # Linear and RBF SVM
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression  # Linear/Logistic Regression
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import GroupKFold
r_state = 45


2. Now we will initialize our full set of ten models and test different hyperparametres

In [2]:
models_and_grids = [
    (KNeighborsClassifier(), {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']}),
    (SVC(), {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}),
    (GaussianProcessClassifier(), {'max_iter_predict': [100, 200]}),
    (DecisionTreeClassifier(), {'max_depth': [None, 10, 20], 'min_samples_split': [2, 5]}),
    (RandomForestClassifier(), {'n_estimators': [50, 100], 'max_depth': [None, 10, 20]}),
    (MLPClassifier(), {'hidden_layer_sizes': [(50,), (100,)], 'learning_rate': ['constant', 'adaptive']}),
    (GaussianNB(), {}),
    (QuadraticDiscriminantAnalysis(), {}),
    (LogisticRegression(), {'C': [0.1, 1, 10], 'solver': ['liblinear', 'lbfgs']}),
    (AdaBoostClassifier(), {'n_estimators': [50, 100], 'learning_rate': [0.5, 1]})
]


3. Test and compare models

In [3]:
num_folds = 9
gkf = GroupKFold(n_splits=num_folds)
best_auc_model = None
best_accuracy_model = None
fastest_models = []
results = []
save_dir = r"C:\Users\fasta\PycharmProjects\binarybird\venv\fully_extracted_data"

# Load only the training data and group information
x_train = np.load(os.path.join(save_dir, 'x_train.npy'))
y_train = np.load(os.path.join(save_dir, 'y_train.npy'))
group_train = np.load(os.path.join(save_dir, 'group_train.npy'))
top_models = []
top_params = []

# Iterate over each model and parameter grid
for model, param_grid in models_and_grids:
    model_name = model.__class__.__name__
    start_time = time.time()
    
    fold_aucs = []
    fold_accuracies = []

    # Set up GroupKFold with GridSearchCV
    grid_search = GridSearchCV(model, param_grid, cv=gkf.split(x_train, y_train, groups=group_train), scoring='accuracy', n_jobs=-1)
    
    try:
        # Fit GridSearchCV on the full training set
        grid_search.fit(x_train, y_train)
        
        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_

        # Track best models and parameters
        top_models.append(best_model)
        top_params.append(best_params)

        # GroupKFold evaluation with the best parameters
        for train_idx, val_idx in gkf.split(x_train, y_train, groups=group_train):
            x_fold_train, x_fold_val = x_train[train_idx], x_train[val_idx]
            y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]

            # Fit on train fold
            best_model.fit(x_fold_train, y_fold_train)
            y_val_pred = best_model.predict(x_fold_val)
            
            # Calculate AUC if applicable
            if hasattr(best_model, "predict_proba"):
                y_val_probs = best_model.predict_proba(x_fold_val)[:, 1]
                auc = roc_auc_score(y_fold_val, y_val_probs) if len(np.unique(y_fold_val)) > 1 else "N/A"
                fold_aucs.append(auc)
            else:
                fold_aucs.append("N/A")

            # Calculate accuracy
            accuracy = accuracy_score(y_fold_val, y_val_pred)
            fold_accuracies.append(accuracy)

        # Average AUC and accuracy across folds
        avg_auc = np.mean([auc for auc in fold_aucs if auc != "N/A"]) if "N/A" not in fold_aucs else "N/A"
        avg_accuracy = np.mean(fold_accuracies)
        run_time = time.time() - start_time

        # Store results for the model
        results.append((model_name, avg_auc, avg_accuracy, run_time, best_params))
        
        # Track the best models by AUROC and accuracy
        if avg_auc != "N/A" and (best_auc_model is None or avg_auc > best_auc_model[1]):
            best_auc_model = (model_name, avg_auc, best_params)
        
        if best_accuracy_model is None or avg_accuracy > best_accuracy_model[1]:
            best_accuracy_model = (model_name, avg_accuracy, best_params)
        
        # Track the three fastest models
        fastest_models.append((model_name, run_time))
        fastest_models = sorted(fastest_models, key=lambda x: x[1])[:3]
        
        print(f"{model_name} completed: Avg AUROC={avg_auc}, Avg Accuracy={avg_accuracy}, Time={run_time:.2f} seconds")
        
    except Exception as e:
        print(f"Error with {model_name}: {e}")
        continue

# Print summary of the top 3 models and their parameters
print("\n--- Top 3 Models by Speed ---")
for model_name, run_time in fastest_models:
    best_params = next((params for (name, _, _, _, params) in results if name == model_name), {})
    print(f"{model_name} - Best Params: {best_params}, Time: {run_time:.2f}s")

# Print the best models by AUROC and accuracy
if best_auc_model:
    print(f"\nBest AUROC Model: {best_auc_model[0]} with AUROC={best_auc_model[1]}, Params={best_auc_model[2]}")
if best_accuracy_model:
    print(f"Best Accuracy Model: {best_accuracy_model[0]} with Accuracy={best_accuracy_model[1]}, Params={best_accuracy_model[2]}")


KNeighborsClassifier completed: Avg AUROC=0.9934393784252094, Avg Accuracy=0.9804933998768269, Time=1.81 seconds
SVC completed: Avg AUROC=N/A, Avg Accuracy=0.9995105237395987, Time=2.37 seconds
GaussianProcessClassifier completed: Avg AUROC=0.5024702498953383, Avg Accuracy=0.9525869800801655, Time=19.20 seconds
DecisionTreeClassifier completed: Avg AUROC=0.9957777506602573, Avg Accuracy=0.9956032634600628, Time=0.17 seconds
RandomForestClassifier completed: Avg AUROC=0.999918896999189, Avg Accuracy=0.9965779410790279, Time=2.42 seconds
MLPClassifier completed: Avg AUROC=0.9991889699918897, Avg Accuracy=0.998531571218796, Time=4.26 seconds
GaussianNB completed: Avg AUROC=0.9991638103849928, Avg Accuracy=0.9975526186979933, Time=0.05 seconds
QuadraticDiscriminantAnalysis completed: Avg AUROC=0.9982624113475178, Avg Accuracy=0.9985358461206335, Time=0.08 seconds
LogisticRegression completed: Avg AUROC=1.0, Avg Accuracy=0.9995105237395987, Time=0.16 seconds


C:\Users\fasta\PycharmProjects\binarybird\venv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\fasta\PycharmProjects\binarybird\venv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\fasta\PycharmProjects\binarybird\venv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\fasta\PycharmProjects\binarybird\venv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed i

AdaBoostClassifier completed: Avg AUROC=1.0, Avg Accuracy=0.9970674173394293, Time=6.12 seconds

--- Top 3 Models by Speed ---
GaussianNB - Best Params: {}, Time: 0.05s
QuadraticDiscriminantAnalysis - Best Params: {}, Time: 0.08s
LogisticRegression - Best Params: {'C': 0.1, 'solver': 'liblinear'}, Time: 0.16s

Best AUROC Model: LogisticRegression with AUROC=1.0, Params={'C': 0.1, 'solver': 'liblinear'}
Best Accuracy Model: SVC with Accuracy=0.9995105237395987, Params={'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}


In [4]:
# Assuming group_train and group_test are your group labels for training and test sets
save_dir = r"C:\Users\fasta\PycharmProjects\binarybird\venv\fully_extracted_data"
group_train = np.load(os.path.join(save_dir, 'group_train.npy'))
group_test = np.load(os.path.join(save_dir, 'group_test.npy'))

# Convert group labels to sets for comparison
train_groups = set(np.unique(group_train))  # Unique group identifiers in the training set
test_groups = set(np.unique(group_test))     # Unique group identifiers in the test set

# Print any common group identifiers
overlap_groups = train_groups.intersection(test_groups)
if overlap_groups:
    print(f"Overlap found between training and test groups: {overlap_groups}")
else:
    print("No overlap between training and test groups.")


No overlap between training and test groups.


4. Results on 

In [5]:
# Load test data
x_test = np.load(os.path.join(save_dir, 'x_test.npy'))
y_test = np.load(os.path.join(save_dir, 'y_test.npy'))

# Initialize a list to hold the results for the top models on the test set
test_results = []

# Iterate over the top 3 models
for model_name, _ in fastest_models:
    # Retrieve the best parameters for the current model
    best_params = next((params for (name, _, _, _, params) in results if name == model_name), {})
    # Initialize the model with the best parameters
    model_class = next((m for m, _ in models_and_grids if m.__class__.__name__ == model_name))
    best_model = model_class.set_params(**best_params)
    # Retrain the model on the entire training set
    best_model.fit(x_train, y_train)
    # Evaluate the model on the test set
    y_test_pred = best_model.predict(x_test)
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_test_pred)
    # Calculate AUROC if applicable
    if hasattr(best_model, "predict_proba"):
        y_test_probs = best_model.predict_proba(x_test)[:, 1]
        auc = roc_auc_score(y_test, y_test_probs) if len(np.unique(y_test)) > 1 else "N/A"
    else:
        auc = "N/A"
    # Store the results for this model
    test_results.append((model_name, auc, accuracy))
    # Print the results
    print(f"{model_name} - Test AUROC: {auc}, Test Accuracy: {accuracy:.4f}")
print("\n--- Summary of Test Results ---")
for model_name, auc, accuracy in test_results:
    print(f"{model_name}: AUROC={auc}, Accuracy={accuracy:.4f}")


GaussianNB - Test AUROC: 0.9981858748462163, Test Accuracy: 0.9984
QuadraticDiscriminantAnalysis - Test AUROC: 1.0, Test Accuracy: 1.0000
LogisticRegression - Test AUROC: 1.0, Test Accuracy: 1.0000

--- Summary of Test Results ---
GaussianNB: AUROC=0.9981858748462163, Accuracy=0.9984
QuadraticDiscriminantAnalysis: AUROC=1.0, Accuracy=1.0000
LogisticRegression: AUROC=1.0, Accuracy=1.0000
